In [1]:
(defclass polynomial ()
  ((poly :accessor poly
         :initform nil
         :initarg :poly)
   (deg :accessor poly-deg
        :initform 0
        :initarg :deg)
   )
)

#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>

In [2]:
;;; Make a polynomial taking advantage of program = data
;;; new polynomial term of the form a * x^b:
(defun make-polynomial (a b)
;;     (print (append b (cons * a)))
     (MAKE-INSTANCE 'polynomial :poly  `(+ (* ,a (expt (x ,b)))) :deg b)
)

MAKE-POLYNOMIAL

In [3]:
;;;; TEST of make-polynomial
(make-polynomial 2 3)

#<POLYNOMIAL {1001FB1AF3}>

In [4]:
(defmethod degree ((p polynomial))
    (labels ( (degree-1 (poly max)
;;                         (print poly)
;;                         (format t "~C max: ~d calc: ~d " #\linefeed max (caddr (caddar poly)) )
                (cond  ( (null poly) max ) 
                        ( (> (caddr (caddar poly)) max) (degree-1 (cdr poly) (caddr (caddar poly))) )
                        (t (degree-1 (cdr poly) max))
                )
               )
             )
            (degree-1 (cdr (slot-value p 'poly)) 0)
     )
    
    
    )

#<STANDARD-METHOD CL-JUPYTER-USER::DEGREE (POLYNOMIAL) {10022A4CD3}>

In [5]:
;;; TESTs of degree
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 7)) (* 3 (expt x 3))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 10)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
      )
     ( print (degree t1))
     ( print (degree t2))
     ( print (degree t3))
     
     )


3 
7 
10 

10

In [144]:
(defmethod print-object ((poly polynomial) out)
    (labels ((to-string (stack term poly  )
;;                 (print "------------------------------------")
;;                 (print stack)
;;                 (print  term)
;;                 (print poly)
                (cond  ((null term) (to-string-1 (reverse stack) "" ))
                       ((atom term) term)
                       ((eql (car term) '*) (to-string (push `(,(to-string stack (cadr term) poly) ,(to-string stack (caddr term) poly) ) stack) (car poly) (cdr poly) ) )
                       ((eql (car term) 'expt) `(,(cadr term),(caddr term)))
;;                        (t (to-string-1 stack))
                 )

              )
              (to-string-1 (stack res )
;;                            (print "AT to-string-1")
;;                            (print stack)
;;                            (format t "res: ~s" res)
;;                            (print (car stack))
                           (let* ((coef (caar stack))
                                 (var  (caadar stack))
                                 (exp  (car (cdadar stack)))
                                 (str  (if (or (null coef) (equal res "") ) (if ( equal res "") (format nil "~:[-~;~]" (>= coef 0)) res) (format nil "~a ~:[- ~;+ ~]" res (>= coef 0))))
                                 )
;;                                  (format t "term: ~a~a~a~C" coef var exp #\linefeed)
;;                                  (when (not (null coef)) (format res "~a ~:[ - ~; + ~]" res (> coef 0)))
                                 (cond 
                                       ((null stack) res );(format t "stack nil "))
                                       ((= coef 0) (to-string-1 (cdr stack) (format nil "~a" res)))
                                       ((= exp 0)  (to-string-1 (cdr stack) (format nil "~a~a" str (abs coef) )))
                                       ((and (= exp 1) (= (abs coef) 1)) (to-string-1 (cdr stack) (format nil "~a~a" str var )))
                                       ((= (abs coef) 1) (to-string-1 (cdr stack) (format nil "~a~a^~a" str var exp ) ) )
                                       ((= exp 1) (to-string-1 (cdr stack) (format nil "~a~a~a" str (abs coef) var)))
                                       (t (to-string-1 (cdr stack) (format nil "~a~a~a^~a" str (abs coef) var exp )))
                                )
                         )
             ) )
           (format out "~a" (to-string '() (cadr (slot-value poly 'poly)) (cddr (slot-value poly 'poly) )  ) )
     )
    
    

      
    )

REDEFINITION-WITH-DEFMETHOD: 
  redefining PRINT-OBJECT (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                           #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD


#<STANDARD-METHOD COMMON-LISP:PRINT-OBJECT (POLYNOMIAL T) {1004C99683}>

In [149]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* -2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3))  
      )
     (print t1)
     (print t2)
     (print t3)
     
     )


X + 2X^2 + 3X^3 
X + 3X^3 + 2X^7 
-2X^2 + 3X^3 + X^10 

-2X^2 + 3X^3 + X^10

In [135]:
(defmethod plus ((p1 polynomial) (p2 polynomial))
    
    (labels ( (plus-1 (p1 p2 res)
                   (let ( (coef1 (cadar p1))
                          (var1  (cadr(caddar p1)))
                          (expt1 (caddr (caddar p1)))
                          (coef2 (cadar p2))
                          (var2  (cadr(caddar p2)))
                          (expt2 (caddr (caddar p2)))
                         )
;;                         (print "-----------------------------------")
;;                         (print res)
;;                         (print p1)
;;                         (print p2)
;;                         (format t "term1: ~a~a^~a" coef1 var1 expt1)
;;                         (format t "term2: ~a~a^~a" coef2 var2 expt2)
                        
                        (cond ((and (null p1) (null p2)) (cons '+ (reverse res)) )
                              ((null p1) (plus-1 nil (cdr p2) (push `(* ,coef2 (expt ,var2 ,expt2)) res)))
                              ((null p2) (plus-1 nil (cdr p1) (push `(* ,coef1 (expt ,var1 ,expt1)) res)))
                              ((= expt1 expt2)  
                                (plus-1 (cdr p1) (cdr p2) (push `(* ,(+ coef1 coef2) (expt ,var1 ,expt1)) res ) ))
                              ((< expt1 expt2) 
                                (plus-1 (cdr p1) p2  (push `(* ,coef1 (expt ,var1 ,expt1)) res)) )
                              ((> expt1 expt2) 
                                (plus-1 p1 (cdr p2)  (push `(* ,coef2 (expt ,var2 ,expt2)) res)) )
                         )
                    )
               ) 
             
             )
            (let ((poly_res (MAKE-INSTANCE 'polynomial ) )
                  (poly (plus-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() ))
                   
                   )
;;                  (print "after: ")
;;                  (print poly)
                 (setf (poly poly_res) poly)
                 (setf (poly-deg  poly_res) (degree poly_res))
;;                  (print (poly poly_res))
                 poly_res
                ) 
;;             (degree poly_res)
            
      )
;;         (print p1)
;;         (print p2)
    
    
)

REDEFINITION-WITH-DEFMETHOD: 
  redefining PLUS (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                   #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::PLUS (POLYNOMIAL POLYNOMIAL) {1005BA0353}>

In [136]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (plus t1 t2))
     ( print (plus t2 t1))
     ( print (plus t1 t3))
     ( print (plus t3 t1))
     ( print (plus t2 t3))
     ( print (plus t3 t2))
     
     )


2X + 2X^2 + 6X^3 + 2X^7 
2X + 2X^2 + 6X^3 + 2X^7 
X + 4X^2 + 6X^3 + X^10 
X + 4X^2 + 6X^3 + X^10 
X + 2X^2 + 6X^3 + 2X^7 + X^10 
X + 2X^2 + 6X^3 + 2X^7 + X^10 

X + 2X^2 + 6X^3 + 2X^7 + X^10

In [157]:
(defmethod minus ((p1 polynomial) (p2 polynomial))
    
    (labels ( (minus-1 (p1 p2 res)
                   (let ( (coef1 (cadar p1))
                          (var1  (cadr(caddar p1)))
                          (expt1 (caddr (caddar p1)))
                          (coef2 (cadar p2))
                          (var2  (cadr(caddar p2)))
                          (expt2 (caddr (caddar p2)))
                         )
;;                         (print "-----------------------------------")
;;                         (print res)
;;                         (print p1)
;;                         (print p2)
;;                         (format t "term1: ~a~a^~a" coef1 var1 expt1)
;;                         (format t "term2: ~a~a^~a" coef2 var2 expt2)
                        
                        (cond ((and (null p1) (null p2)) (cons '+ (reverse res)) )
                              ((null p1) (minus-1 nil (cdr p2) (push `(* ,(- coef2) (expt ,var2 ,expt2)) res)))
                              ((null p2) (minus-1 nil (cdr p1) (push `(* ,coef1 (expt ,var1 ,expt1)) res)))
                              ((= expt1 expt2)  
                                (minus-1 (cdr p1) (cdr p2) (push `(* ,(- coef1 coef2) (expt ,var1 ,expt1)) res ) ))
                              ((< expt1 expt2) 
                                (minus-1 (cdr p1) p2  (push `(* ,coef1 (expt ,var1 ,expt1)) res)) )
                              ((> expt1 expt2) 
                                (minus-1 p1 (cdr p2)  (push `(* ,(- coef2) (expt ,var2 ,expt2)) res)) )
                         )
                    )
               ) 
             
             )
            (let ((poly_res (MAKE-INSTANCE 'polynomial ) )
                  (poly (minus-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() ))
                   
                   )
;;                  (print "after: ")
;;                  (print poly)
                 (setf (poly poly_res) poly)
                 (setf (poly-deg  poly_res) (degree poly_res))
;;                  (print (poly poly_res))
                 poly_res
                ) 
;;             (degree poly_res)
            
      )
;;         (print p1)
;;         (print p2)
    
    
)

REDEFINITION-WITH-DEFMETHOD: 
  redefining MINUS (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                    #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::MINUS (POLYNOMIAL POLYNOMIAL) {10045CFA33}>

In [158]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (minus t1 t2))
     ( print (minus t2 t1))
     ( print (minus t1 t3))
     ( print (minus t3 t1))
     ( print (minus t2 t3))
     ( print (minus t3 t2))
     
     )


2X^2 - 2X^7 
-2X^2 + 2X^7 
X - X^10 
-X + X^10 
X - 2X^2 + 2X^7 - X^10 
-X + 2X^2 - 2X^7 + X^10 

-X + 2X^2 - 2X^7 + X^10

In [ ]:
(defvar *zero*)
(defvar *p1*)
(defvar *p2*)
(defvar *p3*)
(defvar *p4*)

(defvar *test*)

(setq *zero* (make-polynomial 0 0))
(setq *p1* (make-polynomial 4 3))
(setq *p2* (make-polynomial 3 2))
(setq *p3* (make-polynomial 1 0))
(setq *p4* (make-polynomial 2 1))

(setq *test* (make-polynomial 5 5))
;; (make-polynomial 4 3)